In [ ]:
!nvidia-smi

Sun Mar 10 22:10:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install datasets transformers rouge-score nltk py7zr
! pip install wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=084cfaea6a533fd7f02c7ababf84d058ad88f37ca9bb3fef9d7

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [ ]:
model_checkpoint = "facebook/bart-large-xsum"

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-7-a2ef1525156b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content a

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
!pip install transformers[torch]

In [ ]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",  # enable logging to W&B
    run_name="bart-large-xsum-samsum"  # name of the W&B run (optional)
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

wandb: Currently logged in as: eslamashraf-12342 (eslamashraf). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.396300,1.468561,54.222700,29.395500,44.654500,49.869100,30.711500
2,0.847600,1.456757,54.513400,29.460100,44.908100,50.040000,29.824000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

TrainOutput(global_step=5523, training_loss=1.1200658448381784, metrics={'train_runtime': 4274.0884, 'train_samples_per_second': 10.34, 'train_steps_per_second': 1.292, 'total_flos': 2.611682725134336e+16, 'train_loss': 1.1200658448381784, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.4567573070526123,
 'eval_rouge1': 54.5134,
 'eval_rouge2': 29.4601,
 'eval_rougeL': 44.9081,
 'eval_rougeLsum': 50.04,
 'eval_gen_len': 29.824,
 'eval_runtime': 305.6332,
 'eval_samples_per_second': 2.676,
 'eval_steps_per_second': 0.671,
 'epoch': 3.0}

In [ ]:
trainer.save_model("path/to/save/model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


In [ ]:
# Loading the saved model
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("path/to/save/model")
loaded_tokenizer = AutoTokenizer.from_pretrained("path/to/save/model")

In [ ]:
trainer.save_model("/content/drive/MyDrive/GP")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


In [ ]:
import torch
text_to_summarize = '''Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
'''
input_ids = loaded_tokenizer(text_to_summarize, return_tensors="pt")["input_ids"]

with torch.no_grad():
    output = loaded_model.generate(input_ids)

summary = loaded_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(summary)

Amanda can't find Betty's number. Larry called her last time they were at the park. Hannah doesn't know him well, but Amanda thinks he's nice. Hannah will text him.


In [ ]:
import torch
text_to_summarize = '''Assistant : What do you do for a living? Are you a therapist or a psychologist?
User : Not great, to be honest. I've been feeling very low and hopeless lately.
Assistant : I am sorry to hear that. I hope you feel better soon. Are you seeing a psychiatrist?
User : Yes, but I don't see any point in living anymore.
Assistant : I am sorry to hear that. Do you have any family or friends that can help you?
User : I just feel no one loves me. what should I do?
Assistant : I am sorry to hear that you feel that way. I am sure there is someone out there for you.
User : I don't believe you. Nothing ever changes for me. I've tried everything, and nothing works.
Assistant : I am sorry to hear that. That is unfortunate. I am sure you are a good person.
User : That is maybe good. Thank you.
Assistant : You are welcome. There are many types of therapists. You can be a psychologist or a psychiatrist.
User : I don't know. I guess I can try. I have nothing to lose.
Assistant : You have a lot to lose if you don't speak to the patient. You have to be patient.
User : Haha right.
Assistant : Do you have any hobbies that you like to do in your spare time? I like to play video games.
User : Mmmm I like to go clubbing and swim.
Assistant : Swimming is a great way to exercise and relieve stress. You should try it.
User : Yes, swimming changes my mood.
Assistant : Swimming is a great way to exercise and relieve stress. You should try it.
User : Thank you, goodbye.
Assistant : You are welcome. I hope you have a good rest of your day. Goodbye.
'''
input_ids = loaded_tokenizer(text_to_summarize, return_tensors="pt")["input_ids"]

with torch.no_grad():
    output = loaded_model.generate(input_ids)

summary = loaded_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(summary)

User has been feeling low and hopeless lately. He is seeing a psychiatrist, but he doesn't see any point in living anymore. He doesn't have any family or friends that can help him. He likes to play video games, go clubbing and swimming.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer

loaded_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/GP")
loaded_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/GP")

In [ ]:
import torch
text_to_summarize = '''Assistant : What do you do for a living? Are you a therapist or a psychologist?
User : Not great, to be honest. I've been feeling very low and hopeless lately.
Assistant : I am sorry to hear that. I hope you feel better soon. Are you seeing a psychiatrist?
User : Yes, but I don't see any point in living anymore.
Assistant : I am sorry to hear that. Do you have any family or friends that can help you?
User : I just feel no one loves me. what should I do?
Assistant : I am sorry to hear that you feel that way. I am sure there is someone out there for you.
User : I don't believe you. Nothing ever changes for me. I've tried everything, and nothing works.
Assistant : I am sorry to hear that. That is unfortunate. I am sure you are a good person.
User : That is maybe good. Thank you.
Assistant : You are welcome. There are many types of therapists. You can be a psychologist or a psychiatrist.
User : I don't know. I guess I can try. I have nothing to lose.
Assistant : You have a lot to lose if you don't speak to the patient. You have to be patient.
User : Haha right.
Assistant : Do you have any hobbies that you like to do in your spare time? I like to play video games.
User : Mmmm I like to go clubbing and swim.
Assistant : Swimming is a great way to exercise and relieve stress. You should try it.
User : Yes, swimming changes my mood.
Assistant : Swimming is a great way to exercise and relieve stress. You should try it.
User : Thank you, goodbye.
Assistant : You are welcome. I hope you have a good rest of your day. Goodbye.
'''
input_ids = loaded_tokenizer(text_to_summarize, return_tensors="pt")["input_ids"]

with torch.no_grad():
    output = loaded_model.generate(input_ids)

summary = loaded_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(summary)

User has been feeling low and hopeless lately. He is seeing a psychiatrist, but he doesn't see any point in living anymore. He doesn't have any family or friends that can help him. He likes to play video games, go clubbing and swimming.
